In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import os
import numpy as np

pd.set_option("display.precision", 3)

In [ ]:
import random
import numpy as np
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf
        tf.random.set_seed(seed)

set_seed(1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
def pre_proc_comments(text):
  text = text.replace("['", "")
  text = text.rsplit(" [")[0]
  text = text.replace("''", "'").replace("''", "'")
  text = text.replace('""', '"').replace('""', '"')
  text = text.strip()
  return text

def pre_proc_topic(text):
  text = text.replace("[", "")
  text = text.replace("'", "")
  text = text.replace('"', "")
  text = text.strip()
  return text

def generate_no_stance(df_data, topic_name, new_topic_name):
  sub_df = df_data[df_data.topic.isin([topic_name])]
  sub_df.stance.replace('stance', 'neither', inplace=True)
  sub_df.topic = new_topic_name
  sub_df.id = sub_df.id * -1
  return sub_df

file_path = 'https://drive.google.com/uc?export=view&id=1XxfiJA2FeyfS55XUCW827Rb4u0UShb5V'
df_data_orig = pd.read_csv(file_path, header = 0, encoding='utf-8', sep=',', index_col="Unnamed: 0")
df_data_orig.columns = ['id', 'topic', 'comment_orig', 'stance']
df_data_orig.comment_orig = df_data_orig.comment_orig.apply(pre_proc_comments)
df_data_orig.topic = df_data_orig.topic.apply(pre_proc_topic)

df_data_orig['stance_pol'] = df_data_orig.stance
df_data_orig.stance.replace('favor', 'stance', inplace=True)
df_data_orig.stance.replace('against', 'stance', inplace=True)
df_data_orig.stance.replace('not\_related', 'neither', inplace=True)

g1 = generate_no_stance(df_data_orig, 'Contratação', 'Desarmamento')
g2 = generate_no_stance(df_data_orig, 'Servidores Públicos', 'Código Penal')
g3 = generate_no_stance(df_data_orig, 'Reforma Trabalhista', 'Desarmamento')
g4 = generate_no_stance(df_data_orig, 'Desarmamento', 'Servidores Públicos')
concat_df = pd.concat([g1, g2, g3, g4], ignore_index=True)

df_data_orig = pd.concat([df_data_orig, concat_df])

# df_data_orig = df_data_orig[df_data_orig.stance == 'stance']
df_data_orig = df_data_orig[df_data_orig.stance_pol.isin(['favor','against'])]


df = df_data_orig[:]
df['gold_label'] = df['stance_pol']
df['sentence1'] = df['topic']
df['sentence2'] = df['comment_orig']
del df['stance']

train_df = df[df.topic.isin(['Código de Defesa do Consumidor', 'Lei Maria da Penha', 'Porte de Armas', 'Seguro-Desemprego', 'Contratação', 'Estatuto do Desarmamento', 'Reforma Administrativa', 'Reforma Tributária',  'Reforma Trabalhista', 'Desarmamento', 'Reforma Previdenciária',  'Partidos Políticos', 'Código Penal', 'Estatuto da OAB', 'Salário Mínimo', 'LDB'])]

np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

train_df = df_train
val_df = pd.concat([df_val, df_test])

test_df = df[df.topic.isin(['Servidores Públicos', 'Ajuda de custo', 'CLT', 'LOAS' ])]

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-6-1b91767cba50>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [ ]:
len(df[df.topic == 'Servidores Públicos'])

231

In [ ]:
train_df = train_df.dropna()
val_df = val_df.dropna()

In [ ]:
train_df['sentence1'] = train_df['sentence1'].astype(str)
train_df['sentence2'] = train_df['sentence2'].astype(str)

In [ ]:
val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

In [ ]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]

In [ ]:
train_df

,id,topic,comment_orig,stance_pol,gold_label,sentence1,sentence2
1289,82649,Contratação,Se a proposta é aumentar o emprego aos jovens ...,against,against,Contratação,Se a proposta é aumentar o emprego aos jovens ...
1184,11362,Reforma Tributária,"Como sempre, os servidores públicos são coloca...",favor,favor,Reforma Tributária,"Como sempre, os servidores públicos são coloca..."
531,-136641,Desarmamento,"É uma nova reforma trabalhista, a fim de preju...",against,against,Desarmamento,"É uma nova reforma trabalhista, a fim de preju..."
363,-107436,Código Penal,Não vejo nada de positivo em uma medida que qu...,favor,favor,Código Penal,Não vejo nada de positivo em uma medida que qu...
380,84034,Contratação,Dar o poder de escolha as empresas que até a c...,favor,favor,Contratação,Dar o poder de escolha as empresas que até a c...
...,...,...,...,...,...,...,...
279,-112192,Código Penal,Muitos dos servidores públicos não tem salário...,favor,favor,Código Penal,Muitos dos servidores públicos não tem salário...
1109,215131,Reforma Administrativa,"Não é reforma administrativa, é demonte dos se...",against,against,Reforma Administrativa,"Não é reforma administrativa, é demonte dos se..."
497,-22243,Desarmamento,Só para pobres. Tem até reforma trabalhista no...,against,against,Desarmamento,Só para pobres. Tem até reforma trabalhista no...
302,74501,Reforma Tributária,Relevância 0. Reforma tributária e reforma pol...,favor,favor,Reforma Tributária,Relevância 0. Reforma tributária e reforma pol...


In [ ]:
val_df

,id,topic,comment_orig,stance_pol,gold_label,sentence1,sentence2
241,-70650,Desarmamento,A lei precisa ser modernizada. O grande foco n...,favor,favor,Desarmamento,A lei precisa ser modernizada. O grande foco n...
240,209328,Reforma Administrativa,Deveria ser uma reforma política e não adminis...,against,against,Reforma Administrativa,Deveria ser uma reforma política e não adminis...
31,207953,Servidores Públicos,A Reforma Administrativa mantém os privilégios...,favor,favor,Servidores Públicos,A Reforma Administrativa mantém os privilégios...
382,-195517,Código Penal,A estabilidade dos servidores públicos visa a ...,favor,favor,Código Penal,A estabilidade dos servidores públicos visa a ...
480,23993,CLT,"Não trata os privilégios, transição injusta pa...",against,against,CLT,"Não trata os privilégios, transição injusta pa..."
...,...,...,...,...,...,...,...
1294,70105,Contratação,"Não esta clara a vantagem para brasileiros, a ...",favor,favor,Contratação,"Não esta clara a vantagem para brasileiros, a ..."
90,-108512,Desarmamento,Por que o trabalhador tem que pagar essa conta...,favor,favor,Desarmamento,Por que o trabalhador tem que pagar essa conta...
978,212698,LDB,"A lei não respeita o artigo 3o da LDB 9394/97,...",favor,favor,LDB,"A lei não respeita o artigo 3o da LDB 9394/97,..."
273,-111258,Código Penal,"Não concordo com as emendas 4 e 5, propostas p...",favor,favor,Código Penal,"Não concordo com as emendas 4 e 5, propostas p..."


In [ ]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, label_dict={'stance': 0, 'neither': 1}):
    self.label_dict = label_dict

    self.train_df = train_df
    self.val_df = val_df

    self.base_path = '/content/'
    self.tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
    self.train_data = None
    self.val_data = None
    self.init_data()

  def init_data(self):
    # Saving takes too much RAM
    #
    # if os.path.exists(os.path.join(self.base_path, 'train_data.pkl')):
    #   print("Found training data")
    #   with open(os.path.join(self.base_path, 'train_data.pkl'), 'rb') as f:
    #     self.train_data = pickle.load(f)
    # else:
    #   self.train_data = self.load_data(self.train_df)
    #   with open(os.path.join(self.base_path, 'train_data.pkl'), 'wb') as f:
    #     pickle.dump(self.train_data, f)
    # if os.path.exists(os.path.join(self.base_path, 'val_data.pkl')):
    #   print("Found val data")
    #   with open(os.path.join(self.base_path, 'val_data.pkl'), 'rb') as f:
    #     self.val_data = pickle.load(f)
    # else:
    #   self.val_data = self.load_data(self.val_df)
    #   with open(os.path.join(self.base_path, 'val_data.pkl'), 'wb') as f:
    #     pickle.dump(self.val_data, f)
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)

  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader

In [ ]:
mnli_dataset = MNLIDataBert(train_df, val_df, {'against': 0, 'favor': 1})

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

1408
352


In [ ]:
train_loader, val_loader = mnli_dataset.get_data_loaders(batch_size=16)

In [ ]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)
model.to(device)

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 108,924,674 trainable parameters


In [ ]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [ ]:
import time

def train(model, train_loader, val_loader, optimizer, EPOCHS):  
  total_step = len(train_loader)

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
train(model, train_loader, val_loader, optimizer, 10)

Epoch 1: train_loss: 0.4976 train_acc: 0.7628 | val_loss: 0.3141 val_acc: 0.8835
00:00:41.18
Epoch 2: train_loss: 0.2827 train_acc: 0.8970 | val_loss: 0.2409 val_acc: 0.9233
00:00:40.99
Epoch 3: train_loss: 0.1761 train_acc: 0.9361 | val_loss: 0.2011 val_acc: 0.9347
00:00:41.01
Epoch 4: train_loss: 0.0924 train_acc: 0.9716 | val_loss: 0.2730 val_acc: 0.9176
00:00:40.96
Epoch 5: train_loss: 0.0371 train_acc: 0.9901 | val_loss: 0.2853 val_acc: 0.9290
00:00:40.96
Epoch 6: train_loss: 0.0331 train_acc: 0.9922 | val_loss: 0.2128 val_acc: 0.9460
00:00:41.00
Epoch 7: train_loss: 0.0074 train_acc: 0.9979 | val_loss: 0.2488 val_acc: 0.9489
00:00:41.02
Epoch 8: train_loss: 0.0043 train_acc: 0.9993 | val_loss: 0.2649 val_acc: 0.9545
00:00:41.01
Epoch 9: train_loss: 0.0022 train_acc: 0.9993 | val_loss: 0.2686 val_acc: 0.9545
00:00:40.99
Epoch 10: train_loss: 0.0034 train_acc: 0.9986 | val_loss: 0.2699 val_acc: 0.9517
00:00:41.03


In [ ]:
model2 = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)
model2.to(device)
train(model2, train_loader, val_loader, optimizer, 10)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Epoch 1: train_loss: 0.2033 train_acc: 0.9354 | val_loss: 0.3770 val_acc: 0.9119
00:00:45.68
Epoch 2: train_loss: 0.1467 train_acc: 0.9531 | val_loss: 0.3108 val_acc: 0.9176
00:00:43.85
Epoch 3: train_loss: 0.1277 train_acc: 0.9567 | val_loss: 0.2898 val_acc: 0.9233
00:00:44.57
Epoch 4: train_loss: 0.1018 train_acc: 0.9709 | val_loss: 0.3624 val_acc: 0.9006
00:00:44.15
Epoch 5: train_loss: 0.0786 train_acc: 0.9780 | val_loss: 0.4068 val_acc: 0.8835
00:00:44.23
Epoch 6: train_loss: 0.0780 train_acc: 0.9766 | val_loss: 0.2920 val_acc: 0.9375
00:00:44.24
Epoch 7: train_loss: 0.0533 train_acc: 0.9844 | val_loss: 0.3182 val_acc: 0.9347
00:00:44.18
Epoch 8: train_loss: 0.0468 train_acc: 0.9879 | val_loss: 0.4277 val_acc: 0.9148
00:00:44.18
Epoch 9: train_loss: 0.0513 train_acc: 0.9808 | val_loss: 0.3931 val_acc: 0.9261
00:00:44.12
Epoch 10: train_loss: 0.0644 train_acc: 0.9837 | val_loss: 0.2735 val_acc: 0.9233
00:00:44.20


In [ ]:
model2 = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)
model2.to(device)

mnli_dataset = MNLIDataBert(train_df, val_df, {'against': 0, 'for': 1})
train_loader, val_loader = mnli_dataset.get_data_loaders(batch_size=32)

train(model2, train_loader, val_loader, optimizer, 10)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

1335
334
Epoch 1: train_loss: 0.6900 train_acc: 0.5487 | val_loss: 0.6966 val_acc: 0.4907
00:00:17.53
Epoch 2: train_loss: 0.6874 train_acc: 0.5556 | val_loss: 0.6968 val_acc: 0.4980
00:00:17.64
Epoch 3: train_loss: 0.6887 train_acc: 0.5367 | val_loss: 0.6959 val_acc: 0.4980
00:00:17.45
Epoch 4: train_loss: 0.6883 train_acc: 0.5431 | val_loss: 0.6956 val_acc: 0.5016
00:00:17.34
Epoch 5: train_loss: 0.6916 train_acc: 0.5586 | val_loss: 0.6984 val_acc: 0.4907
00:00:17.39
Epoch 6: train_loss: 0.6886 train_acc: 0.5514 | val_loss: 0.6981 val_acc: 0.4907
00:00:17.48
Epoch 7: train_loss: 0.6892 train_acc: 0.5511 | val_loss: 0.6972 val_acc: 0.4907
00:00:17.50
Epoch 8: train_loss: 0.6909 train_acc: 0.5499 | val_loss: 0.6961 val_acc: 0.5016
00:00:17.48
Epoch 9: train_loss: 0.6880 train_acc: 0.5598 | val_loss: 0.6960 val_acc: 0.5053
00:00:17.48
Epoch 10: train_loss: 0.6890 train_acc: 0.5539 | val_loss: 0.6980 val_acc: 0.4797
00:00:17.43


In [ ]:
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer

result = {}

classifiers = {'BERT1': model}


for i, y_test in test_df.reset_index().groupby('topic'):
  # del y_test['index']
  result[i] = {}

  for c_name, c_classifier in classifiers.items():
    
    ######
    outputs = []
    outputs_prob = []
    test_labels = []

    mnli_dataset_ = MNLIDataBert(y_test, y_test, {'against': 0, 'favor': 1})

    test_dataloader, _ = mnli_dataset_.get_data_loaders(batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model_ = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        # for test_input, test_label in test_dataloader:
        for pair_token_ids, mask_ids, seg_ids, y in test_dataloader:

              # test_label = test_label.to(device)
              # mask = test_input['attention_mask'].to(device)
              # input_id = test_input['input_ids'].squeeze(1).to(device)

              # output = model_(input_id, mask)

              #####
              pair_token_ids = pair_token_ids.to(device)
              mask_ids = mask_ids.to(device)
              seg_ids = seg_ids.to(device)
              # print('>>>',y)
              labels = y.to(device)
              output = model_(pair_token_ids, mask_ids, seg_ids)
              # loss, output = model_(pair_token_ids, 
              #                       token_type_ids=seg_ids, 
              #                       attention_mask=mask_ids, 
              #                       labels=labels).values()
              ####
              # print(output)
              prob = output.logits #nnf.linear(output, dim=1)


              # top_p, top_class = prob.topk(1, dim = 1)
              outputs_prob.append(tuple(prob.cpu().detach().numpy()))

              outputs.append(tuple(output.logits.argmax(dim=1).cpu().detach().numpy()))
              test_labels.append(tuple(y.cpu().detach().numpy()))

              # print(output.logits.argmax(dim=1).cpu(),999, y.cpu())

              acc = (output.logits.argmax(dim=1).cpu() == y).sum().item()
              total_acc_test += acc

    
    print(f'Test Accuracy: {total_acc_test / len(y_test): .3f}')

    #####

    predicted, predicted_prob, test_labels = outputs, outputs_prob, test_labels
    # predicted = np.asarray(predicted, dtype=tuple)
    print(test_labels)
    print(predicted)
    # preds = predicted[list(y_test.index)]
    # m = metrics.f1_score(MultiLabelBinarizer().fit_transform(test_labels),
    #                      MultiLabelBinarizer().fit_transform(predicted),
    #                      average='weighted')
    print(len(y_test), len(test_labels), len(predicted))
    m = metrics.f1_score(test_labels, predicted, average='weighted')
    result[i]["f1-score "+c_name] = m
    
  result[i]["suport"]= len(y_test)

pd.DataFrame(result).T

44
44
Test Accuracy:  0.886
[(0,), (1,), (0,), (1,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (0,), (0,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,)]
[(0,), (1,), (0,), (1,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,)]
44 44 44
72
72
Test Accuracy:  1.000
[(1,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0

,f1-score BERT1,suport
Ajuda de custo,0.887,44.0
CLT,1.000,72.0
LOAS,0.889,18.0
Servidores Públicos,1.000,231.0


In [ ]:
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer

result = {}

# model3 = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
# model3.load_state_dict(torch.load('/content/drive/MyDrive/bertmodels/sent_entail_bert_sd_classify.pt'))
# model3.eval()

classifiers = {'BERT1': model}

outcomes_pred = []
outcomes_test = []
for i, y_test in test_df.reset_index().groupby('sentence1'):
  # del y_test['index']
  result[i] = {}

  for c_name, c_classifier in classifiers.items():
    
    ######
    outputs = []
    outputs_prob = []
    test_labels = []

    mnli_dataset_ = MNLIDataBert(y_test, y_test, {'against': 0, 'favor': 1})

    test_dataloader, _ = mnli_dataset_.get_data_loaders(batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model_ = c_classifier.cuda()

    total_acc_test = 0
    with torch.no_grad():

        # for test_input, test_label in test_dataloader:
        for pair_token_ids, mask_ids, seg_ids, y in test_dataloader:

              pair_token_ids = pair_token_ids.to(device)
              mask_ids = mask_ids.to(device)
              seg_ids = seg_ids.to(device)
              
              labels = y.to(device)
              output = model_(pair_token_ids, mask_ids, seg_ids)

              prob = output.logits #nnf.linear(output, dim=1)


              outputs_prob.append(tuple(prob.cpu().detach().numpy()))

              outputs.append(tuple(output.logits.argmax(dim=1).cpu().detach().numpy()))
              test_labels.append(tuple(y.cpu().detach().numpy()))

              acc = (output.logits.argmax(dim=1).cpu() == y).sum().item()
              total_acc_test += acc

    
    print(f'Test Accuracy: {total_acc_test / len(y_test): .3f}')

    #####

    predicted, predicted_prob, test_labels = outputs, outputs_prob, test_labels
    # predicted = np.asarray(predicted, dtype=tuple)
    print(test_labels)
    print(predicted)
    outcomes_pred.append(predicted)
    outcomes_test.append(test_labels)
    # preds = predicted[list(y_test.index)]
    # m = metrics.f1_score(MultiLabelBinarizer().fit_transform(test_labels),
    #                      MultiLabelBinarizer().fit_transform(predicted),
    #                      average='weighted')
    print(len(y_test), len(test_labels), len(predicted))
    
    vals = metrics.f1_score(test_labels, predicted, labels=[0,  1], average=None)
    result[i]["f1 macro "+c_name] = sum(vals) / 2.0
    result[i]["f1 0"+c_name] = vals[0]
    result[i]["f1 1"+c_name] = vals[1]

    
  result[i]["suport"]= len(y_test)

pd.DataFrame(result).T

44
44
Test Accuracy:  0.886
[(0,), (1,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (1,), (0,)]
[(0,), (1,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (1,), (0,)]
44 44 44
72
72
Test Accuracy:  1.000
[(0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0

,f1 macro BERT1,f1 0BERT1,f1 1BERT1,suport
Ajuda de custo,0.875,0.912,0.839,44.0
CLT,1.000,1.000,1.000,72.0
LOAS,0.883,0.857,0.909,18.0
Servidores Públicos,1.000,1.000,1.000,231.0


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
score_dict = {}
labels_dict = [0,  1]
def compute_scores(score_fn, true_labels, pred_labels, class_wise, name):
        '''
        Computes scores using the given scoring function of the given name. The scores
        are stored in the internal score dictionary.
        :param score_fn: the scoring function to use.
        :param true_labels: the true labels.
        :param pred_labels: the predicted labels.
        :param class_wise: flag to determine whether to compute class-wise scores in
                            addition to macro-averaged scores.
        :param name: the name of this score function, to be used in storing the scores.
        '''
        n = float(len([0,  1]))

        vals = score_fn(true_labels, pred_labels, labels=[0,  1], average=None)
        score_dict['{}_macro'.format(name)] = sum(vals) / n
        if class_wise:
            score_dict['{}_anti'.format(name)] = vals[0]
            score_dict['{}_pro'.format(name)] = vals[1]
            if n > 2:
                score_dict['{}_none'.format(name)] = vals[2]


compute_scores(f1_score, 
               [item for sublist in outcomes_test for item in sublist], 
               [item for sublist in outcomes_pred for item in sublist], True, 'f')
print(score_dict)

{'f_macro': 0.9795208438534477, 'f_anti': 0.9743589743589743, 'f_pro': 0.9846827133479211}


In [ ]:
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer

result = {}

classifiers = {'BERT1': model2}


for i, y_test in test_df.reset_index().groupby('topic'):
  # del y_test['index']
  result[i] = {}

  for c_name, c_classifier in classifiers.items():
    
    ######
    outputs = []
    outputs_prob = []
    test_labels = []

    mnli_dataset_ = MNLIDataBert(y_test, y_test, {'against': 0, 'for': 1})

    test_dataloader, _ = mnli_dataset_.get_data_loaders(batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model_ = c_classifier.cuda()

    total_acc_test = 0
    with torch.no_grad():

        # for test_input, test_label in test_dataloader:
        for pair_token_ids, mask_ids, seg_ids, y in test_dataloader:

              # test_label = test_label.to(device)
              # mask = test_input['attention_mask'].to(device)
              # input_id = test_input['input_ids'].squeeze(1).to(device)

              # output = model_(input_id, mask)

              #####
              pair_token_ids = pair_token_ids.to(device)
              mask_ids = mask_ids.to(device)
              seg_ids = seg_ids.to(device)
              # print('>>>',y)
              labels = y.to(device)
              output = model_(pair_token_ids, mask_ids, seg_ids)
              # loss, output = model_(pair_token_ids, 
              #                       token_type_ids=seg_ids, 
              #                       attention_mask=mask_ids, 
              #                       labels=labels).values()
              ####
              # print(output)
              prob = output.logits #nnf.linear(output, dim=1)


              # top_p, top_class = prob.topk(1, dim = 1)
              outputs_prob.append(tuple(prob.cpu().detach().numpy()))

              outputs.append(tuple(output.logits.argmax(dim=1).cpu().detach().numpy()))
              test_labels.append(tuple(y.cpu().detach().numpy()))

              # print(output.logits.argmax(dim=1).cpu(),999, y.cpu())

              acc = (output.logits.argmax(dim=1).cpu() == y).sum().item()
              total_acc_test += acc

    
    print(f'Test Accuracy: {total_acc_test / len(y_test): .3f}')

    #####

    predicted, predicted_prob, test_labels = outputs, outputs_prob, test_labels
    # predicted = np.asarray(predicted, dtype=tuple)
    print(test_labels)
    print(predicted)
    # preds = predicted[list(y_test.index)]
    # m = metrics.f1_score(MultiLabelBinarizer().fit_transform(test_labels),
    #                      MultiLabelBinarizer().fit_transform(predicted),
    #                      average='weighted')
    print(len(y_test), len(test_labels), len(predicted))
    m = metrics.f1_score(test_labels, predicted, average='weighted')
    result[i]["f1-score "+c_name] = m
    
  result[i]["suport"]= len(y_test)

pd.DataFrame(result).T

604
604
Test Accuracy:  0.439
[(0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (1,

,f1-score BERT1,suport
cota,0.370598,604.0
legalização das drogas,0.578978,516.0


In [ ]:
res1 = [
[
[(0,), (0,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (0,), (0,), (1,)],
[(0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (1,)]
],[
[(0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,)],
[(0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,)]
],[
[(1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (1,)],
[(1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (1,), (1,)]
],[
[(0,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,)],
[(0,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,)]
]]

res2 = [
[
[(1,), (0,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (1,)],
[(1,), (0,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (1,)]
],[
[(0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (0,), (0,), (1,)],
[(0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (0,), (0,), (1,)]
],[
[(1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (0,), (1,), (1,), (1,), (0,), (1,)],
[(1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,)]
],[
[(0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,)],
[(1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (1,), (0,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,)]
]]

from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix


for r in res1:
  print(confusion_matrix(r[0], r[1]))

[[26  3]
 [ 2 13]]
[[55  0]
 [ 0 17]]
[[ 6  1]
 [ 0 11]]
[[ 45   1]
 [  1 184]]


In [ ]:
for r in res2:
  print(confusion_matrix(r[0], r[1]))

[[25  4]
 [ 3 12]]
[[52  3]
 [ 2 15]]
[[ 2  5]
 [ 0 11]]
[[ 31  15]
 [  4 181]]


In [ ]:
torch.save(model.state_dict(), 'sent_entail_bert_sd_classify.pt')

In [ ]:
model3 = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model3.load_state_dict(torch.load('sent_entail_bert_sd_identify.pt'))
model3.eval()

In [ ]:
def stancer(file_path, topic):
  df_data_orig = pd.read_csv(file_path, header = 0, encoding='latin_1', sep=';')

  df_data_orig['topic'] = topic

  df_data_orig.stance.replace('yes', 'stance', inplace=True)
  df_data_orig.stance.replace('no', 'neither', inplace=True)

  df_data_orig['category'] = df_data_orig['stance']
  return df_data_orig

df = pd.concat([stancer('/content/drive/MyDrive/mestrado/Pesquisa_Camara_Posicionamento/Mtwitter-abortion.csv', 'aborto'),
                stancer('/content/drive/MyDrive/mestrado/Pesquisa_Camara_Posicionamento/Mtwitter-age.csv', 'maioridade penal'),
                stancer('/content/drive/MyDrive/mestrado/Pesquisa_Camara_Posicionamento/Mtwitter-death.csv', 'pena de morte'),
                stancer('/content/drive/MyDrive/mestrado/Pesquisa_Camara_Posicionamento/Mtwitter-drugs.csv', 'legalização das drogas'),
                stancer('/content/drive/MyDrive/mestrado/Pesquisa_Camara_Posicionamento/Mtwitter-quotas.csv', 'cota')])

df = df[['topic', 'text', 'polarity', 'category']]


df['gold_label'] = df['polarity']
df['sentence1'] = df['topic']
df['sentence2'] = df['text']

df = df[df['category'] == 'stance']


train_df = df[df.topic.isin(['aborto', 'idade penal', 'Reforma Administrativa', 'pena de morte'])]
test_df = df[df.topic.isin(['legalização das drogas', 'cota'])]

print(len(train_df.topic.value_counts()), len(test_df.topic.value_counts()))

np.random.seed(112)
train_df, val_df = np.split(train_df.sample(frac=1, random_state=42), 
                                     [int(.8*len(train_df))])

print(len(train_df),len(val_df), len(test_df))

train_df

2 2
1335 334 1120


,topic,text,polarity,category,gold_label,sentence1,sentence2
755,pena de morte,É o assunto do momento doa a quem doer. sou a ...,for,stance,for,pena de morte,É o assunto do momento doa a quem doer. sou a ...
893,pena de morte,Especialista da onu condena retomada de pena d...,against,stance,against,pena de morte,Especialista da onu condena retomada de pena d...
1394,aborto,Arquivo do blog!: Aborto: só defende quem está...,against,stance,against,aborto,Arquivo do blog!: Aborto: só defende quem está...
204,pena de morte,"além disso, essa pena de morte só afetaria o p...",against,stance,against,pena de morte,"além disso, essa pena de morte só afetaria o p..."
2487,pena de morte,Quais crimes merecem a pena de morte? sequestr...,for,stance,for,pena de morte,Quais crimes merecem a pena de morte? sequestr...
...,...,...,...,...,...,...,...
65,pena de morte,a pena de Morte já existe. Milhares morrem em ...,for,stance,for,pena de morte,a pena de Morte já existe. Milhares morrem em ...
1912,pena de morte,Pena de morte é pouco pra esses vagabundos que...,for,stance,for,pena de morte,Pena de morte é pouco pra esses vagabundos que...
951,pena de morte,Estado americano de Maryland proíbe pena de morte,against,stance,against,pena de morte,Estado americano de Maryland proíbe pena de morte
2460,pena de morte,Quais crimes merecem a pena de morte? Pedofili...,for,stance,for,pena de morte,Quais crimes merecem a pena de morte? Pedofili...


In [ ]:
test_df

,id,topic,comment_orig,stance_pol,gold_label,sentence1,sentence2
0,104802,Servidores Públicos,Até 50% dos servidores públicos. Famílias depe...,favor,favor,Servidores Públicos,Até 50% dos servidores públicos. Famílias depe...
1,104820,Servidores Públicos,A maioria dos servidores públicos recebe menos...,favor,favor,Servidores Públicos,A maioria dos servidores públicos recebe menos...
2,104840,Servidores Públicos,Diminua o salário e benefícios dos políticos a...,favor,favor,Servidores Públicos,Diminua o salário e benefícios dos políticos a...
3,104879,Servidores Públicos,Onde está a contribuição dos servidores públic...,against,against,Servidores Públicos,Onde está a contribuição dos servidores públic...
5,110091,Servidores Públicos,O estado deveria taxar as grandes fortunas ao ...,favor,favor,Servidores Públicos,O estado deveria taxar as grandes fortunas ao ...
...,...,...,...,...,...,...,...
605,-59796,Servidores Públicos,Ha que mudar a lei do desarmamento totalmente....,against,against,Servidores Públicos,Ha que mudar a lei do desarmamento totalmente....
606,-74505,Servidores Públicos,"esse é um direito do cidadao de bem, esta lei ...",against,against,Servidores Públicos,"esse é um direito do cidadao de bem, esta lei ..."
607,-74521,Servidores Públicos,Devemos de respeitar o plebiscito e a lei do d...,favor,favor,Servidores Públicos,Devemos de respeitar o plebiscito e a lei do d...
608,-74778,Servidores Públicos,Regras mais claras para o tema. Reflete a real...,against,against,Servidores Públicos,Regras mais claras para o tema. Reflete a real...


In [ ]:
from sklearn import metrics

result = {}

for i, y_test in test_df.reset_index().groupby('topic'):
  y_test = y_test["gold_label"]
  result[i] = {}

  m = metrics.f1_score(y_test, ['against']*len(y_test), average='weighted')
  result[i]["f1-score 0 "] = m

  m = metrics.f1_score(y_test, ['favor']*len(y_test), average='weighted')
  result[i]["f1-score 1 "] = m
    
  result[i]["suport"]= len(y_test)

pd.DataFrame(result).T

,f1-score 0,f1-score 1,suport
Ajuda de custo,0.523661,0.173344,44.0
CLT,0.661636,0.090200,72.0
LOAS,0.217778,0.463602,18.0
Servidores Públicos,0.066138,0.712309,231.0


In [ ]:
result = {}

classifiers = {'BERT1': model}

t = pd.concat([train_df, val_df, test_df])
for i, y_test in t.reset_index().groupby('topic'):
  # del y_test['index']
  result[i] = {}

  for c_name, c_classifier in classifiers.items():
    
    ######
    outputs = []
    outputs_prob = []
    test_labels = []

    mnli_dataset_ = MNLIDataBert(y_test, y_test, {'against': 0, 'for': 1})

    test_dataloader, _ = mnli_dataset_.get_data_loaders(batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model_ = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        # for test_input, test_label in test_dataloader:
        for pair_token_ids, mask_ids, seg_ids, y in test_dataloader:

              # test_label = test_label.to(device)
              # mask = test_input['attention_mask'].to(device)
              # input_id = test_input['input_ids'].squeeze(1).to(device)

              # output = model_(input_id, mask)

              #####
              pair_token_ids = pair_token_ids.to(device)
              mask_ids = mask_ids.to(device)
              seg_ids = seg_ids.to(device)
              # print('>>>',y)
              labels = y.to(device)
              output = model_(pair_token_ids, mask_ids, seg_ids)
              # loss, output = model_(pair_token_ids, 
              #                       token_type_ids=seg_ids, 
              #                       attention_mask=mask_ids, 
              #                       labels=labels).values()
              ####
              # print(output)
              prob = output.logits #nnf.linear(output, dim=1)


              # top_p, top_class = prob.topk(1, dim = 1)
              outputs_prob.append(tuple(prob.cpu().detach().numpy()))

              outputs.append(tuple(output.logits.argmax(dim=1).cpu().detach().numpy()))
              test_labels.append(tuple(y.cpu().detach().numpy()))

              # print(output.logits.argmax(dim=1).cpu(),999, y.cpu())

              acc = (output.logits.argmax(dim=1).cpu() == y).sum().item()
              total_acc_test += acc

    
    print(f'Test Accuracy: {total_acc_test / len(y_test): .3f}')

    #####

    predicted, predicted_prob, test_labels = outputs, outputs_prob, test_labels
    # predicted = np.asarray(predicted, dtype=tuple)
    print(test_labels)
    print(predicted)
    # preds = predicted[list(y_test.index)]
    # m = metrics.f1_score(MultiLabelBinarizer().fit_transform(test_labels),
    #                      MultiLabelBinarizer().fit_transform(predicted),
    #                      average='weighted')
    print(len(y_test), len(test_labels), len(predicted))
    m = metrics.f1_score(test_labels, predicted, average='weighted')
    result[i]["f1-score "+c_name] = m
    
  result[i]["suport"]= len(y_test)

pd.DataFrame(result).T

624
624
Test Accuracy:  0.457
[(0,), (0,), (1,), (1,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (1,), (0,), (0,), (0,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (0,), (0,), (1,

,f1-score BERT1,suport
aborto,0.413713,624.0
cota,0.552941,604.0
legalização das drogas,0.522407,516.0
pena de morte,0.586566,1045.0


In [ ]:
torch.save(model.state_dict(), 'sent_entail_bertimbau_sd_classify.pt')